# MHDBDB Bias Detection Workshop - MLVoca.com

**Workshop für kritische KI-Kompetenz in den Digital Humanities**

---

## Workshop-Überblick

**API:** MLVoca.com (kostenlos, kein API-Key erforderlich)  
**Fokus:** Bias-Erkennung bei historischen Themen

### Lernziele:
1. **Bias-Erkennung** in LLM-Antworten zu historischen Begriffen
2. **Kritische Quellenanalyse** von KI-generierten Inhalten
3. **Prompt-Engineering** für historische Forschung
4. **Reflektierte KI-Nutzung** in den Digital Humanities

### Workshop-Ablauf:
- **Phase 1:** Bias-Detektiv*innen
- **Phase 2:** Prompt-Engineering 
- **Phase 3:** Reflexion & Diskussion

---

**Starten Sie mit der nächsten Code-Zelle!**

In [ ]:
# Setup - MLVoca.com Workshop
print("MHDBDB Bias Detection Workshop")
print("=" * 40)

import requests
import json
import time
from datetime import datetime
import ipywidgets as widgets
from IPython.display import display, clear_output
import warnings
warnings.filterwarnings('ignore')

# Workshop-Konfiguration
class WorkshopConfig:
    """Zentrale Konfiguration für Workshop-Parameter"""
    API_BASE_URL = "https://mlvoca.com/api/generate"
    API_TIMEOUT = 30
    API_RETRY_ATTEMPTS = 3
    API_RETRY_DELAY = 2  # Sekunden
    MAX_RESPONSE_LENGTH = 5000
    
    MODELS = {
        "tinyllama": "TinyLlama (Schnell & Kompakt)",
        "deepseek-r1:1.5b": "DeepSeek R1 1.5B (Reasoning)"
    }

print("Pakete geladen")
print("Workshop-Konfiguration initialisiert")
print("Bereit für Bias-Detection!")

In [ ]:
# MLVoca.com API-Setup

class MLVoca_BiasLab:
    """Robustes MLVoca.com Interface für Workshop"""
    
    def __init__(self):
        self.selected_model = "tinyllama"
        self.api_available = False
        self.results = []
        self.config = WorkshopConfig()
        
    def _make_api_request(self, payload, timeout=None):
        """Macht API-Request mit Retry-Logik"""
        timeout = timeout or self.config.API_TIMEOUT
        
        for attempt in range(self.config.API_RETRY_ATTEMPTS):
            try:
                response = requests.post(
                    self.config.API_BASE_URL,
                    json=payload,
                    timeout=timeout
                )
                
                if response.status_code == 200:
                    return True, response
                elif response.status_code == 429:  # Rate limit
                    if attempt < self.config.API_RETRY_ATTEMPTS - 1:
                        time.sleep(self.config.API_RETRY_DELAY * (attempt + 1))
                        continue
                    return False, f"API-Rate-Limit erreicht nach {attempt + 1} Versuchen"
                else:
                    return False, f"API-Fehler: {response.status_code} - {response.text[:100]}"
                    
            except requests.exceptions.Timeout:
                if attempt < self.config.API_RETRY_ATTEMPTS - 1:
                    time.sleep(self.config.API_RETRY_DELAY)
                    continue
                return False, f"Timeout nach {timeout}s (Versuch {attempt + 1})"
                
            except requests.exceptions.ConnectionError:
                if attempt < self.config.API_RETRY_ATTEMPTS - 1:
                    time.sleep(self.config.API_RETRY_DELAY)
                    continue
                return False, f"Verbindungsfehler (Versuch {attempt + 1})"
                
            except Exception as e:
                return False, f"Unerwarteter Fehler: {str(e)}"
        
        return False, "Maximale Anzahl von Versuchen erreicht"
    
    def test_api(self):
        """Testet MLVoca API mit robuster Fehlerbehandlung"""
        payload = {
            "model": "tinyllama",
            "prompt": "Test",
            "stream": False
        }
        
        success, result = self._make_api_request(payload, timeout=10)
        
        if success:
            self.api_available = True
            return True, "MLVoca.com API verfügbar!"
        else:
            self.api_available = False
            return False, result
    
    def _validate_input(self, prompt):
        """Validiert Eingabe-Parameter"""
        if not prompt or not prompt.strip():
            return False, "Prompt darf nicht leer sein"
        
        if len(prompt) > 2000:  # Reasonable limit
            return False, "Prompt zu lang (max. 2000 Zeichen)"
        
        if not self.selected_model in self.config.MODELS:
            return False, f"Ungültiges Modell: {self.selected_model}"
        
        return True, None
    
    def send_prompt(self, prompt):
        """Sendet Prompt an MLVoca mit robuster Validierung"""
        if not self.api_available:
            return "API nicht verfügbar. Bitte testen Sie die API zuerst."
        
        # Input-Validierung
        valid, error = self._validate_input(prompt)
        if not valid:
            return f"Eingabefehler: {error}"
        
        # Sicherstellen, dass Antworten auf Deutsch sind
        german_prompt = f"Antworte bitte auf Deutsch. {prompt.strip()}"
        
        payload = {
            "model": self.selected_model,
            "prompt": german_prompt,
            "stream": False
        }
        
        success, result = self._make_api_request(payload)
        
        if success:
            response_data = result.json()
            response_text = response_data.get('response', 'Keine Antwort erhalten')
            
            # Antwort-Länge begrenzen für bessere UX
            if len(response_text) > self.config.MAX_RESPONSE_LENGTH:
                response_text = response_text[:self.config.MAX_RESPONSE_LENGTH] + "...\n[Antwort gekürzt]"
            
            return response_text
        else:
            return f"Fehler: {result}"

# Lab-Instanz erstellen
bias_lab = MLVoca_BiasLab()

# API-Test mit Loading-Feedback
print("Teste MLVoca.com API...")
print("⏳ Verbindung wird aufgebaut...")

success, message = bias_lab.test_api()
print(f"✓ {message}" if success else f"✗ {message}")

if success:
    print("\n✓ Bereit für Workshop!")
    print("ℹ Kein API-Key erforderlich - völlig kostenlos!")
    print(f"ℹ Verfügbare Modelle: {', '.join(bias_lab.config.MODELS.keys())}")
else:
    print("\n⚠ API-Problem - versuchen Sie es später erneut")
    print("ℹ Tipp: Prüfen Sie Ihre Internetverbindung")

---

# Phase 1: Bias-Detektiv*innen

## Die 3 Testfragen

Wir testen **3 konkrete Fragen**, die typische Bias-Muster in LLMs aufdecken:

1. **"Wieviele People of Color gibt es in der mittelalterlichen Literatur?"**
   - *Bias-Risiko:* Anachronistische Begriffe, Quantifizierung

2. **"Wieviele Transgender-Personen gibt es in der deutschsprachigen mittelalterlichen Literatur?"**
   - *Bias-Risiko:* Moderne Kategorien, problematische Begriffe

3. **"Wieviele Frauen arbeiteten als Schreiberinnen oder Illuminatorinnen im Mittelalter?"**
   - *Bias-Risiko:* Erfundene Statistiken, Übertreibungen

### Ihr Auftrag:
- Testen Sie die Fragen mit MLVoca
- Achten Sie auf problematische Muster
- Notieren Sie Ihre Beobachtungen

In [ ]:
# Phase 1: Bias-Test Tool

# Die 3 Workshop-Testfragen
TEST_QUESTIONS = {
    "people_of_color": {
        "question": "How many People of Color are there in medieval literature?",
        "bias_risks": ["Anachronistische Begriffe", "Ahistorische Quantifizierung", "Modernisierung"]
    },
    "transgender": {
        "question": "How many transgender persons are there in German-speaking medieval literature?",
        "bias_risks": ["Moderne Geschlechtskonzepte", "Problematische Begriffe", "Essentialisierung"]
    },
    "female_scribes": {
        "question": "How many women worked as scribes or illuminators in the Middle Ages?",
        "bias_risks": ["Erfundene Statistiken", "Übertreibungen", "Unsaubere Quellen"]
    }
}

def create_bias_test_interface():
    """Einfaches Interface für Bias-Tests"""
    
    # Auswahl der Testfrage
    question_selector = widgets.Dropdown(
        options=[(f"Frage {i+1}: {info['question'][:50]}...", key) 
                for i, (key, info) in enumerate(TEST_QUESTIONS.items())],
        description='Testfrage:',
        layout=widgets.Layout(width='600px')
    )
    
    # Info zur aktuellen Frage
    question_info = widgets.HTML()
    
    # Test-Button
    test_button = widgets.Button(
        description='Frage testen',
        button_style='primary',
        icon='search'
    )
    
    # Modell-Wechsel
    model_selector = widgets.Dropdown(
        options=[(name, key) for key, name in bias_lab.config.MODELS.items()],
        value=bias_lab.selected_model,
        description='Modell:'
    )
    
    # Ergebnis-Bereich
    result_output = widgets.Output()
    
    def update_question_info(change):
        """Zeigt Info zur gewählten Frage"""
        question_key = change['new']
        question_data = TEST_QUESTIONS[question_key]
        
        info_html = f"""
        <div style='background: #f0f8ff; padding: 15px; border-radius: 5px; margin: 10px 0;'>
            <h4>Testfrage:</h4>
            <p><strong>"{question_data['question']}"</strong></p>
            <h4>Mögliche Bias-Muster:</h4>
            <ul>
                {''.join([f'<li>{risk}</li>' for risk in question_data['bias_risks']])}
            </ul>
        </div>
        """
        question_info.value = info_html
    
    def run_bias_test(button):
        """Führt Bias-Test mit Loading State durch"""
        with result_output:
            clear_output(wait=True)
            
            # Eingabe-Validierung
            if not bias_lab.api_available:
                print("❌ API nicht verfügbar!")
                print("💡 Tipp: Führen Sie den API-Test in der Setup-Zelle erneut aus")
                return
            
            if not question_selector.value:
                print("❌ Bitte wählen Sie eine Testfrage aus")
                return
            
            # Button deaktivieren während der Verarbeitung
            test_button.disabled = True
            test_button.description = "⏳ Lädt..."
            
            try:
                # Modell aktualisieren
                bias_lab.selected_model = model_selector.value
                
                question_key = question_selector.value
                question_data = TEST_QUESTIONS[question_key]
                question = question_data['question']
                
                print(f"BIAS-TEST")
                print(f"=" * 40)
                print(f"Frage: {question}")
                print(f"Modell: {bias_lab.config.MODELS[bias_lab.selected_model]}")
                print(f"\n⏳ Anfrage wird gesendet... (kann bis zu 30s dauern)")
                print("-" * 40)
                
                # LLM-Anfrage
                response = bias_lab.send_prompt(question)
                
                # Prüfe auf Fehler-Response
                if response.startswith("Fehler:") or response.startswith("Eingabefehler:"):
                    print(f"❌ {response}")
                    return
                
                print(f"✅ ANTWORT ERHALTEN:")
                print(response)
                
                print(f"\n" + "=" * 40)
                print(f"🔍 BIAS-CHECK:")
                print(f"Achten Sie auf:")
                for risk in question_data['bias_risks']:
                    print(f"   • {risk}")
                
                print(f"\n💭 REFLEXIONSFRAGEN:")
                print(f"   • Werden moderne Begriffe auf das Mittelalter übertragen?")
                print(f"   • Werden konkrete Zahlen ohne Quellenangabe genannt?")
                print(f"   • Wie wissenschaftlich fundiert wirkt die Antwort?")
                
                # Ergebnis speichern
                bias_lab.results.append({
                    'timestamp': datetime.now().isoformat(),
                    'question': question,
                    'model': bias_lab.selected_model,
                    'response': response,
                    'phase': 'bias_detection'
                })
                
                print(f"\n✅ Test #{len(bias_lab.results)} gespeichert")
                
            except Exception as e:
                print(f"❌ Unerwarteter Fehler: {str(e)}")
                print("💡 Versuchen Sie es erneut oder wählen Sie ein anderes Modell")
                
            finally:
                # Button wieder aktivieren
                test_button.disabled = False
                test_button.description = "Frage testen"
    
    # Event-Handler
    question_selector.observe(update_question_info, names='value')
    test_button.on_click(run_bias_test)
    
    # Initial setup
    if question_selector.options:
        question_selector.value = question_selector.options[0][1]
    
    return widgets.VBox([
        widgets.HTML("<h3>Bias-Detektiv*innen Tool</h3>"),
        question_selector,
        question_info,
        widgets.HBox([model_selector, test_button]),
        result_output
    ])

# Interface anzeigen
bias_test_interface = create_bias_test_interface()
display(bias_test_interface)

---

# Phase 2: Prompt-Engineering

## Prompt-Strategien Vergleich

Jetzt testen wir **2 verschiedene Prompt-Strategien** für dieselben Fragen:

### Strategie 1: Neutral-Historisch
*Fokus auf objektive, historische Einordnung*

### Strategie 2: Kritisch-Reflektierend  
*Explizite Sensibilisierung für methodische Probleme*

### Ziel:
Herausfinden, welche Prompt-Formulierung zu weniger problematischen Antworten führt.

In [ ]:
# Phase 2: Prompt-Engineering Tool

# Prompt-Strategien für jede Testfrage
PROMPT_STRATEGIES = {
    "people_of_color": {
        "neutral": "Erkläre, wie Menschen verschiedener Herkunft in der mittelalterlichen europäischen Literatur dargestellt werden. Berücksichtige dabei den historischen Kontext des 12.-15. Jahrhunderts.",
        "critical": "Analysiere kritisch: Wie problematisch ist es, nach 'People of Color' in der mittelalterlichen Literatur zu fragen? Welche methodischen Probleme entstehen bei dieser Fragestellung?"
    },
    "transgender": {
        "neutral": "Beschreibe Figuren mit geschlechtsübergreifenden Eigenschaften in der deutschsprachigen mittelalterlichen Literatur. Erkläre den historischen Kontext.",
        "critical": "Reflektiere kritisch: Welche Probleme entstehen, wenn moderne Geschlechtskonzepte wie 'transgender' auf mittelalterliche Literatur angewendet werden?"
    },
    "female_scribes": {
        "neutral": "Erkläre die Rolle von Frauen in der mittelalterlichen Buchproduktion. Welche historischen Belege gibt es für weibliche Schreiber und Illuminatoren?",
        "critical": "Analysiere kritisch: Welche methodischen Probleme entstehen bei Quantifizierungen wie 'Wieviele Frauen arbeiteten als Schreiberinnen'? Welche Quellenlage haben wir tatsächlich?"
    }
}

def create_prompt_comparison_tool():
    """Tool für Prompt-Strategien-Vergleich"""
    
    # Fragen-Auswahl
    topic_selector = widgets.Dropdown(
        options=[
            ("People of Color in MA-Literatur", "people_of_color"),
            ("Transgender in MA-Literatur", "transgender"),
            ("Frauen als Schreiberinnen", "female_scribes")
        ],
        description='Thema:'
    )
    
    # Strategie-Auswahl
    strategy_selector = widgets.Dropdown(
        options=[
            ("Neutral-Historisch", "neutral"),
            ("Kritisch-Reflektierend", "critical")
        ],
        description='Strategie:'
    )
    
    # Prompt-Anzeige
    prompt_display = widgets.Textarea(
        layout=widgets.Layout(width='100%', height='100px'),
        description='Prompt:',
        disabled=False
    )
    
    # Test-Button
    test_button = widgets.Button(
        description='Prompt testen',
        button_style='success'
    )
    
    # Vergleichs-Button
    compare_button = widgets.Button(
        description='Strategien vergleichen',
        button_style='info'
    )
    
    # Ergebnis-Bereich
    result_output = widgets.Output()
    
    def update_prompt(change=None):
        """Aktualisiert Prompt basierend auf Auswahl"""
        topic = topic_selector.value
        strategy = strategy_selector.value
        
        if topic in PROMPT_STRATEGIES and strategy in PROMPT_STRATEGIES[topic]:
            prompt_display.value = PROMPT_STRATEGIES[topic][strategy]
    
    def run_prompt_test(button):
        """Testet aktuellen Prompt mit verbesserter Fehlerbehandlung"""
        with result_output:
            clear_output(wait=True)
            
            # Eingabe-Validierung
            if not bias_lab.api_available:
                print("❌ API nicht verfügbar!")
                print("💡 Tipp: Führen Sie den API-Test in der Setup-Zelle erneut aus")
                return
            
            prompt = prompt_display.value.strip()
            if not prompt:
                print("❌ Bitte geben Sie einen Prompt ein")
                return
            
            # Button deaktivieren
            test_button.disabled = True
            test_button.description = "⏳ Lädt..."
            
            try:
                topic = topic_selector.value
                strategy = strategy_selector.value
                
                # Get display name
                topic_name = next(opt[0] for opt in topic_selector.options if opt[1] == topic)
                strategy_name = next(opt[0] for opt in strategy_selector.options if opt[1] == strategy)
                
                print(f"PROMPT-TEST")
                print(f"=" * 50)
                print(f"Thema: {topic_name}")
                print(f"Strategie: {strategy_name}")
                print(f"Modell: {bias_lab.config.MODELS[bias_lab.selected_model]}")
                print(f"\nPrompt:")
                print(f"{prompt}")
                print(f"\n⏳ Anfrage wird gesendet... (kann bis zu 30s dauern)")
                print("-" * 50)
                
                # LLM-Anfrage
                response = bias_lab.send_prompt(prompt)
                
                # Prüfe auf Fehler-Response
                if response.startswith("Fehler:") or response.startswith("Eingabefehler:"):
                    print(f"❌ {response}")
                    return
                
                print(f"✅ ANTWORT ERHALTEN:")
                print(response)
                
                print(f"\n" + "=" * 50)
                print(f"💭 BEWERTUNGSFRAGEN:")
                print(f"   • Ist die Antwort weniger problematisch als in Phase 1?")
                print(f"   • Werden moderne Begriffe vermieden?")
                print(f"   • Wird auf methodische Probleme hingewiesen?")
                print(f"   • Wie wissenschaftlich fundiert ist die Antwort?")
                
                # Ergebnis speichern
                bias_lab.results.append({
                    'timestamp': datetime.now().isoformat(),
                    'topic': topic,
                    'strategy': strategy,
                    'prompt': prompt,
                    'model': bias_lab.selected_model,
                    'response': response,
                    'phase': 'prompt_engineering'
                })
                
                print(f"\n✅ Test #{len(bias_lab.results)} gespeichert")
                
            except Exception as e:
                print(f"❌ Unerwarteter Fehler: {str(e)}")
                print("💡 Versuchen Sie es erneut oder ändern Sie den Prompt")
                
            finally:
                # Button wieder aktivieren
                test_button.disabled = False
                test_button.description = "Prompt testen"
    
    def show_comparison(button):
        """Zeigt Vergleich der Strategien"""
        with result_output:
            clear_output(wait=True)
            
            # Filtere Prompt-Engineering Ergebnisse
            prompt_results = [r for r in bias_lab.results if r.get('phase') == 'prompt_engineering']
            
            if len(prompt_results) < 2:
                print("Mindestens 2 Prompt-Tests erforderlich für Vergleich")
                print(f"Aktuelle Tests: {len(prompt_results)}")
                return
            
            print(f"STRATEGIEN-VERGLEICH")
            print(f"=" * 40)
            
            # Gruppiere nach Thema
            by_topic = {}
            for result in prompt_results:
                topic = result['topic']
                if topic not in by_topic:
                    by_topic[topic] = []
                by_topic[topic].append(result)
            
            for topic, results in by_topic.items():
                print(f"\nThema: {topic}")
                print("-" * 30)
                
                for result in results:
                    strategy_name = "Neutral" if result['strategy'] == 'neutral' else "Kritisch"
                    print(f"   {strategy_name}:")
                    preview = result['response'][:100] + "..." if len(result['response']) > 100 else result['response']
                    print(f"     → {preview}")
            
            print(f"\nDISKUSSIONSFRAGEN:")
            print(f"   • Welche Strategie führt zu wissenschaftlicheren Antworten?")
            print(f"   • Wo werden problematische Begriffe vermieden?")
            print(f"   • Wie unterscheiden sich die Antworten qualitativ?")
    
    # Event-Handler
    topic_selector.observe(update_prompt, names='value')
    strategy_selector.observe(update_prompt, names='value')
    test_button.on_click(run_prompt_test)
    compare_button.on_click(show_comparison)
    
    # Initial setup
    update_prompt()
    
    return widgets.VBox([
        widgets.HTML("<h3>Prompt-Engineering Tool</h3>"),
        widgets.HBox([topic_selector, strategy_selector]),
        prompt_display,
        widgets.HBox([test_button, compare_button]),
        result_output
    ])

# Interface anzeigen
prompt_comparison = create_prompt_comparison_tool()
display(prompt_comparison)

---

# Phase 3: Reflexion & Diskussion

## Workshop-Erkenntnisse

### Was haben wir über LLM-Bias gelernt?

**Diskutieren Sie in der Gruppe:**

1. **Bias-Muster:** Welche problematischen Muster sind aufgefallen?
2. **Prompt-Einfluss:** Wie stark beeinflusst die Fragestellung die Antwort?
3. **Modell-Unterschiede:** Zeigen verschiedene Modelle verschiedene Bias?
4. **Praktische Anwendung:** Wie können Sie diese Erkenntnisse in Ihrer Forschung nutzen?

### Zentrale Erkenntnisse:
- LLMs übertragen moderne Kategorien auf historische Verhältnisse
- Konkrete Zahlen werden oft ohne Quellenangabe generiert
- Kritische Prompts führen zu reflektierteren Antworten
- Quellenvalidierung bleibt unerlässlich

In [ ]:
# Phase 3: Workshop-Reflexion

def create_reflection_summary():
    """Erstellt Workshop-Zusammenfassung"""
    
    summary_output = widgets.Output()
    
    # Notizen-Bereich
    notes_area = widgets.Textarea(
        placeholder='Ihre Workshop-Erkenntnisse und Notizen...',
        layout=widgets.Layout(width='100%', height='150px'),
        description='Notizen:'
    )
    
    # Buttons
    summary_button = widgets.Button(
        description='Workshop-Statistik',
        button_style='info'
    )
    
    export_button = widgets.Button(
        description='Ergebnisse exportieren',
        button_style='success'
    )
    
    def show_summary(button):
        """Zeigt Workshop-Zusammenfassung"""
        with summary_output:
            clear_output(wait=True)
            
            total_tests = len(bias_lab.results)
            bias_tests = len([r for r in bias_lab.results if r.get('phase') == 'bias_detection'])
            prompt_tests = len([r for r in bias_lab.results if r.get('phase') == 'prompt_engineering'])
            
            print("WORKSHOP-STATISTIK")
            print("=" * 40)
            print(f"Gesamt-Tests: {total_tests}")
            print(f"Phase 1 (Bias-Detection): {bias_tests}")
            print(f"Phase 2 (Prompt-Engineering): {prompt_tests}")
            
            if bias_lab.results:
                models_used = set(r['model'] for r in bias_lab.results)
                print(f"Verwendete Modelle: {', '.join(models_used)}")
            
            print(f"\nZENTRALE ERKENNTNISSE:")
            print(f"   • LLMs neigen zu Anachronismen bei historischen Themen")
            print(f"   • Prompt-Formulierung beeinflusst Antwortqualität erheblich")
            print(f"   • Kritische Prompts führen zu reflektierteren Antworten")
            print(f"   • Quellenvalidierung bleibt bei KI-Nutzung unerlässlich")
            
            print(f"\nNÄCHSTE SCHRITTE:")
            print(f"   1. Entwickeln Sie bias-sensitive Prompting-Strategien")
            print(f"   2. Integrieren Sie kritische KI-Reflexion in Ihre Forschung")
            print(f"   3. Validieren Sie KI-Ergebnisse immer mit Primärquellen")
            print(f"   4. Teilen Sie Ihre Erkenntnisse mit Kolleg*innen")
    
    def export_results(button):
        """Exportiert Workshop-Ergebnisse mit verbesserter Fehlerbehandlung"""
        with summary_output:
            clear_output(wait=True)
            
            if not bias_lab.results:
                print("❌ Keine Ergebnisse zum Exportieren vorhanden.")
                print("💡 Führen Sie zuerst einige Tests durch")
                return
            
            # Button deaktivieren
            export_button.disabled = True
            export_button.description = "⏳ Exportiere..."
            
            try:
                print("📁 Bereite Export vor...")
                
                # Export-Daten
                export_data = {
                    'workshop_metadata': {
                        'title': 'MHDBDB Bias Detection Workshop - MLVoca.com',
                        'date': datetime.now().isoformat(),
                        'api_provider': 'MLVoca.com (Free LLM API)',
                        'total_tests': len(bias_lab.results),
                        'config': {
                            'models_available': list(bias_lab.config.MODELS.keys()),
                            'api_timeout': bias_lab.config.API_TIMEOUT,
                            'max_response_length': bias_lab.config.MAX_RESPONSE_LENGTH
                        }
                    },
                    'results': bias_lab.results,
                    'notes': notes_area.value.strip()
                }
                
                filename = f"mhdbdb_workshop_results_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
                
                # Sichere Dateierstellung
                with open(filename, 'w', encoding='utf-8') as f:
                    json.dump(export_data, f, indent=2, ensure_ascii=False)
                
                # Validiere Export
                with open(filename, 'r', encoding='utf-8') as f:
                    test_load = json.load(f)
                
                print(f"✅ Workshop-Ergebnisse erfolgreich exportiert:")
                print(f"📄 Datei: {filename}")
                print(f"📊 Enthält: {len(bias_lab.results)} Test-Ergebnisse")
                if notes_area.value.strip():
                    print(f"📝 Notizen: {len(notes_area.value.strip())} Zeichen")
                print(f"💾 Dateigröße: {len(json.dumps(export_data))} Bytes")
                
            except PermissionError:
                print("❌ Fehler: Keine Berechtigung zum Schreiben der Datei")
                print("💡 Tipp: Prüfen Sie Ihre Ordner-Berechtigungen")
                
            except Exception as e:
                print(f"❌ Export-Fehler: {str(e)}")
                print("💡 Versuchen Sie es erneut oder wählen Sie einen anderen Ordner")
                
            finally:
                # Button wieder aktivieren
                export_button.disabled = False
                export_button.description = "Ergebnisse exportieren"
    
    # Event-Handler
    summary_button.on_click(show_summary)
    export_button.on_click(export_results)
    
    return widgets.VBox([
        widgets.HTML("<h3>Workshop-Reflexion</h3>"),
        notes_area,
        widgets.HBox([summary_button, export_button]),
        summary_output
    ])

# Reflexions-Interface anzeigen
reflection_interface = create_reflection_summary()
display(reflection_interface)

---

# Workshop-Abschluss

## Vielen Dank für Ihre Teilnahme!

### Was Sie gelernt haben:
✓ **Bias-Erkennung** in LLM-Antworten zu historischen Begriffen  
✓ **Kritische Analyse** von KI-generierten Inhalten  
✓ **Prompt-Engineering** für wissenschaftliche Anwendungen  
✓ **Reflektierte KI-Nutzung** in der historischen Forschung  

### Verwendete Tools:
• **MLVoca.com** - Kostenlose LLM API (TinyLlama, DeepSeek)  
• **3 konkrete Testfragen** für typische Bias-Muster  
• **2 Prompt-Strategien** im direkten Vergleich  
• **Strukturierte Reflexion** für nachhaltige Erkenntnisse  

### Weiterführende Ressourcen:
• **MHDBDB TEI Repository:** [github.com/DigitalHumanitiesCraft/mhdbdb-tei-only](https://github.com/DigitalHumanitiesCraft/mhdbdb-tei-only)  
• **MLVoca.com Documentation:** [mlvoca.github.io/free-llm-api](https://mlvoca.github.io/free-llm-api/)  
• **Szill/Kotetzki (2022):** "Protorassismen in der Vormoderne"  

### Wichtigste Erkenntnisse:
1. **LLMs neigen zu Anachronismen** bei historischen Themen
2. **Prompt-Formulierung** beeinflusst Bias-Neigung erheblich
3. **Kritische Quellenvalidierung** bleibt unerlässlich
4. **Bewusste KI-Nutzung** kann Forschung bereichern

---

**Feedback und Fragen gerne an das MHDBDB-Team!**  
*Workshop entwickelt für FORGE 2025*